In [33]:
import pandas as pd
from pandas import Series
from pandas import DataFrame
import numpy as np
from pathlib import Path

data_file = Path(r'C:\Users\Fabian\Dropbox\CS 767\Project\labor-conflict-data.csv')
data = pd.read_csv(filepath_or_buffer=data_file)
data = data.pivot(index='MonthYear', columns='QuadClass', values='f0_')
rng = pd.date_range(start='1/1/1999', periods=data.shape[0], freq='M')     # date index
data.index = rng
data.columns = ['Verbal_Coop', 'Material_Coop', 'Verbal_Conflict', 'Material_Conflict']
print(data.head())

            Verbal_Coop  Material_Coop  Verbal_Conflict  Material_Conflict
1999-01-31           19              5                4                  1
1999-02-28           21              6                2                  1
1999-03-31           19              5                6                 14
1999-04-30           21              4               13                  3
1999-05-31           10              4                4                  7


In [6]:
%matplotlib
import matplotlib.pyplot as plt

data.plot()
plt.show()

Using matplotlib backend: TkAgg


In [12]:
data.describe()

,Verbal_Coop,Material_Coop,Verbal_Conflict,Material_Conflict
count,237.000000,237.000000,237.000000,237.000000
mean,1891.704641,479.046414,478.405063,402.540084
std,2155.652427,537.435769,543.659559,453.367984
min,9.000000,2.000000,2.000000,1.000000
25%,36.000000,9.000000,13.000000,14.000000
50%,665.000000,186.000000,170.000000,133.000000
75%,3621.000000,916.000000,919.000000,777.000000
max,7196.000000,1818.000000,1847.000000,1395.000000


In [19]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(Series(data['Material_Coop']), lags=50)
plt.show()


In [23]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(Series(data['Verbal_Conflict']), lags=50)
plt.show()

In [31]:
from statsmodels.tsa.arima_model import ARIMA
n_fit = data['Material_Conflict'].shape[0]//3*2
model = ARIMA(Series(data['Material_Conflict'])[:n_fit], order=(2,1,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())


                              ARIMA Model Results                              
Dep. Variable:     D.Material_Conflict   No. Observations:                  157
Model:                  ARIMA(2, 1, 1)   Log Likelihood                -878.400
Method:                        css-mle   S.D. of innovations             65.019
Date:                 Tue, 09 Oct 2018   AIC                           1766.801
Time:                         19:29:26   BIC                           1782.082
Sample:                     02-28-1999   HQIC                          1773.007
                          - 02-29-2012                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         3.8443      2.235      1.720      0.088      -0.537       8.226
ar.L1.D.Material_Conflict     0.4417      0.145      3.038      0.003       0.

In [34]:
residuals = DataFrame(model_fit.resid)
residuals.plot()
plt.show()


In [36]:
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())


                0
count  157.000000
mean    -0.050900
std     65.227871
min   -287.007361
25%    -14.149991
50%     -8.425270
75%      3.042754
max    456.888029


In [43]:
X = Series(data['Material_Conflict']).values
size = len(X) // 3 * 2
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(2,1,1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    #print('predicted=%f, expected=%f' % (yhat, obs))
error = np.square(np.subtract(test, predictions)).mean()
print('Test RMSE: %.3f' % np.sqrt(error))
# plot
data_line, = plt.plot(test, label="Data")
pred_line, = plt.plot(predictions, color='red', label="Predictions")
plt.legend(handles=[data_line, pred_line])
plt.show()


Test RMSE: 358.812
